In [1]:
from utils.sample import Sample
from scipy.optimize import brentq
import pandas as pd
import numpy as np
from utils.biseccion import bisec
from utils.black_scholes import raiz_ratio
from sklearn.metrics import r2_score

In [5]:
opn = Sample(ratio=[0.4, 1.6], T=[0.2, 1.1], r=[0.02, 0.1], o=[0.01, 1])

opn.create('prueba', N=10**5)
x_test, y_test = opn.open('prueba')

prueba is done ...


In [6]:
df = pd.DataFrame(x_test, columns=['c/k', 'ratio', 'r', 'T'])
df['o'] = y_test
df.head()

,c/k,ratio,r,T,o
0,0.122994,0.960710,0.043287,0.790269,0.369813
1,0.007790,0.821175,0.048923,0.779890,0.157504
2,0.000194,0.495399,0.057878,0.370634,0.416265
3,0.196376,0.784737,0.085064,1.090094,0.740476
4,0.534926,1.411525,0.073874,0.368848,0.917778


In [7]:
# Tiro los casos en que no se puede aplicar el metodo de biseccion
# y calculo la volatilidad implícita
vol_bisec = []
vol_brent = []
i = 0
drops = []
for c, ratio, r, T in x_test:
    f = lambda x: raiz_ratio(c, ratio, r, x, T)
    # que se cumpla la precondicción
    if f(0.01) < 0:
        # máxima precision
        o_bic = bisec(f, 0.01, 1, 2**-56)
        o_bren = brentq(f, 0.01, 1)
        vol_bisec.append(o_bic)
        vol_brent.append(o_bren)
    else: 
        drops.append(i)
    i += 1
    
    if i % 10000 == 0:
        print('{}%'.format(100*i/len(x_test)))
    
    
dfb = df.drop(drops)
#volatilidad implicita
dfb['o_bis'] = vol_bisec
dfb['o_bren'] = vol_brent

10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%
100.0%


In [8]:
# error entre la volatilidad estimada e implícita
dfb['diff_bis'] = (dfb['o'] - dfb['o_bis']).apply(abs)
dfb['diff_bren'] = (dfb['o'] - dfb['o_bren']).apply(abs)
print('Error cuadratico medio biseccion: ', np.square(dfb['o'] - dfb['o_bis']).mean())
print('Error absoluto medio biseccion: ', dfb['diff_bis'].mean())
aux = 100*(np.abs(dfb['o'] - dfb['o_bis']) / dfb['o']).mean() 
print('Error absoluto porcentual medio biseccion', aux)
print('r2 Biseccion', r2_score(dfb['o'], dfb['o_bis']))

print('\n\n')

print('Error cuadratico medio Brent: ', np.square(dfb['o'] - dfb['o_bren']).mean())
print('Error absoluto medio Brent: ', dfb['diff_bren'].mean())
aux = 100*(np.abs(dfb['o'] - dfb['o_bren']) / dfb['o']).mean() 
print('Error absoluto porcentual medio Brent', aux)
print('r2 Brent', r2_score(dfb['o'], dfb['o_bren']))

Error cuadratico medio biseccion:  1.239505855668202e-08
Error absoluto medio biseccion:  2.98127468262937e-06
Error absoluto porcentual medio biseccion 0.00462372919498589
r2 Biseccion 0.9999998422675915



Error cuadratico medio Brent:  5.504440883918925e-06
Error absoluto medio Brent:  7.882653498811466e-05
Error absoluto porcentual medio Brent 0.3067899070840565
r2 Brent 0.9999299536412686


In [9]:
#Ordeno el dataframe segun el error entre la volatilidad estimada
# y la volatilidad implícita del método de bisección
orda = dfb.sort_values('diff_bis', ignore_index=True, ascending= False)

In [10]:
#aplico la función que busca la raiz sobre la volatilidad generada
#mediante el método de bisección
fs = []
auxi = orda[['c/k','ratio','r', 'o_bis', 'T']]
for i in range(len(auxi)):
    fs.append(raiz_ratio(*auxi.iloc[i]))

orda['f(o_bis)'] = fs


In [11]:
orda.head(50)

,c/k,ratio,r,T,o,o_bis,o_bren,diff_bis,diff_bren,f(o_bis)
0,0.593291,1.575640,0.080051,0.222473,0.121159,0.132844,0.130742,0.011685,0.009583,0.0
1,0.581479,1.565731,0.071183,0.222989,0.128955,0.118281,0.128594,0.010674,0.000362,0.0
2,0.592465,1.572687,0.071264,0.280305,0.107745,0.118281,0.118292,0.010536,0.010547,0.0
3,0.540697,1.527552,0.027907,0.474118,0.076353,0.083477,0.080093,0.007123,0.003740,0.0
4,0.423593,1.389915,0.086283,0.397051,0.069400,0.076346,0.072395,0.006947,0.002995,0.0
5,0.550895,1.543773,0.025510,0.280206,0.103812,0.110547,0.109979,0.006735,0.006167,0.0
6,0.478757,1.463709,0.029865,0.507718,0.067186,0.073809,0.071940,0.006622,0.004753,0.0
7,0.590991,1.578463,0.041559,0.303370,0.108305,0.102813,0.102815,0.005492,0.005490,0.0
8,0.592711,1.518806,0.073845,1.039725,0.058651,0.064141,0.064150,0.005489,0.005499,0.0
9,0.482456,1.453732,0.087329,0.333733,0.086002,0.091211,0.089090,0.005209,0.003088,0.0


Observar q en muchos casos la diferencia entre la volatilidad implícita y la volatilidad estimada es grande, eso es porque en ciertos casos la volatilidad tiene poco impacto, osea un "vega bajo", y por el problema de precisión la función que calcula la raíz retorna 0.

In [12]:
import scipy.stats as si
#vega en funcion de ratio osea la función retorna vega/K.
def vega(ratio, r, o, T):
    
    d1 = (np.log(ratio) + (r + 0.5 * o ** 2) * T) / (o * np.sqrt(T))
    
    vega = ratio * si.norm.cdf(d1, 0.0, 1.0) * np.sqrt(T)
    
    return vega

In [13]:
def my_vega(elem):
    return vega(elem['ratio'], elem['r'], elem['o'], elem['T'])

orda['vega/K'] = orda.apply(my_vega, axis=1)

In [14]:
orda.head(50)

,c/k,ratio,r,T,o,o_bis,o_bren,diff_bis,diff_bren,f(o_bis),vega/K
0,0.593291,1.575640,0.080051,0.222473,0.121159,0.132844,0.130742,0.011685,0.009583,0.0,0.743182
1,0.581479,1.565731,0.071183,0.222989,0.128955,0.118281,0.128594,0.010674,0.000362,0.0,0.739365
2,0.592465,1.572687,0.071264,0.280305,0.107745,0.118281,0.118292,0.010536,0.010547,0.0,0.832640
3,0.540697,1.527552,0.027907,0.474118,0.076353,0.083477,0.080093,0.007123,0.003740,0.0,1.051814
4,0.423593,1.389915,0.086283,0.397051,0.069400,0.076346,0.072395,0.006947,0.002995,0.0,0.875813
5,0.550895,1.543773,0.025510,0.280206,0.103812,0.110547,0.109979,0.006735,0.006167,0.0,0.817189
6,0.478757,1.463709,0.029865,0.507718,0.067186,0.073809,0.071940,0.006622,0.004753,0.0,1.042956
7,0.590991,1.578463,0.041559,0.303370,0.108305,0.102813,0.102815,0.005492,0.005490,0.0,0.869402
8,0.592711,1.518806,0.073845,1.039725,0.058651,0.064141,0.064150,0.005489,0.005499,0.0,1.548679
9,0.482456,1.453732,0.087329,0.333733,0.086002,0.091211,0.089090,0.005209,0.003088,0.0,0.839815


In [15]:
orda.tail()

,c/k,ratio,r,T,o,o_bis,o_bren,diff_bis,diff_bren,f(o_bis),vega/K
97975,0.314199,0.975415,0.095347,0.946904,0.788987,0.788987,0.788987,0.0,7.771561e-16,0.0,0.645892
97976,0.008136,0.444536,0.030083,0.522646,0.699992,0.699992,0.699992,0.0,9.992007e-16,0.0,0.030132
97977,0.606297,1.347031,0.062687,0.846023,0.923181,0.923181,0.923181,0.0,3.330669e-16,0.0,0.989886
97978,0.583777,1.400358,0.042624,0.963924,0.705110,0.705110,0.705110,0.0,1.313394e-13,0.0,1.118832
97979,0.026767,0.714132,0.040895,0.506334,0.477510,0.477510,0.477510,0.0,1.665335e-16,0.0,0.113629


In [16]:
#porcentaje de elementos en que no se encontro la raiz
# mediante el método de bisección con tolerancia 2**-64
100*len(orda[orda['f(o_bis)'] < 0])/len(orda)

7.171871810573586